In [8]:
from azureml.core import Workspace, Experiment, ScriptRunConfig, Environment
from azureml.train.hyperdrive import HyperDriveConfig, PrimaryMetricGoal
from azureml.train.hyperdrive import RandomParameterSampling, BanditPolicy, uniform, choice

# Load the workspace
ws = Workspace.from_config()

# Define the hyperparameter search space
param_sampling = RandomParameterSampling({
    "learning_rate": uniform(0.01, 0.1),
    "num_leaves": choice(20, 30, 40, 50),
    "max_depth": choice(5, 10, 15),
    "min_child_samples": choice(10, 20, 30),
    "reg_alpha": uniform(0, 1),
    "reg_lambda": uniform(0, 1)
})

# Define the early termination policy
early_termination_policy = BanditPolicy(
    slack_factor=0.1,
    evaluation_interval=1,
    delay_evaluation=5
)

# Create the environment (replace with your actual environment name and details)
env = Environment.get(workspace=ws, name="lightgbm-environment")

# Define the script to run
script_run_config = ScriptRunConfig(
    source_directory=".",
    script="train_lightgbm.py",  # Script to train LightGBM
    compute_target="cpu-cluster",  # Replace with your compute cluster
    environment=env  # Use the custom environment
)

# Configure HyperDrive
hyperdrive_config = HyperDriveConfig(
    run_config=script_run_config,  # Use script_run_config instead of run_config
    hyperparameter_sampling=param_sampling,
    policy=early_termination_policy,
    primary_metric_name="AUC",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,
    max_concurrent_runs=4
)

# Submit the HyperDrive run
experiment = Experiment(ws, "lightgbm-hyperdrive-optimization")
hyperdrive_run = experiment.submit(hyperdrive_config)
hyperdrive_run.wait_for_completion(show_output=True)


RunId: HD_b5537f09-c817-46e3-b246-5fb8e0c86fa0
Web View: https://ml.azure.com/runs/HD_b5537f09-c817-46e3-b246-5fb8e0c86fa0?wsid=/subscriptions/b2303599-e8d0-4d67-bdeb-dd94f4970c1d/resourcegroups/frauddetectionrg/workspaces/frauddetectionml&tid=4df09b9c-261e-4691-89d8-c6fcd95ee9f9

Streaming azureml-logs/hyperdrive.txt

[2025-02-09T22:09:09.7296767Z][GENERATOR][DEBUG]Sampled 4 jobs from search space 
[2025-02-09T22:09:10.2134743Z][SCHEDULER][INFO]Scheduling job, id='HD_b5537f09-c817-46e3-b246-5fb8e0c86fa0_0' 
[2025-02-09T22:09:10.2684166Z][SCHEDULER][INFO]Scheduling job, id='HD_b5537f09-c817-46e3-b246-5fb8e0c86fa0_1' 
[2025-02-09T22:09:10.4196499Z][SCHEDULER][INFO]Scheduling job, id='HD_b5537f09-c817-46e3-b246-5fb8e0c86fa0_2' 
[2025-02-09T22:09:10.4186201Z][SCHEDULER][INFO]Scheduling job, id='HD_b5537f09-c817-46e3-b246-5fb8e0c86fa0_3' 
[2025-02-09T22:09:12.2251902Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_b5537f09-c817-46e3-b246-5fb8e0c86fa0_1' 
[2025-02-09T22:09:12.2687194

{'runId': 'HD_b5537f09-c817-46e3-b246-5fb8e0c86fa0',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2025-02-09T22:09:08.05333Z',
 'endTimeUtc': '2025-02-09T22:21:18.110842Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"AUC","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '4d5e7fb5-422b-4403-81c7-b1ed3655b177',
  'user_agent': 'python/3.10.11 (Linux-5.15.0-1073-azure-x86_64-with-glibc2.31) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.59.0',
  'best_child_run_id': 'HD_b5537f09-c817-46e3-b246-5fb8e0c86fa0_9',
  'score': '0.9834425927095656',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_b5537f09-c817-46e3-b246-5fb8e0c86fa0_9'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': 'azureml-sdk-train',

In [11]:
from azureml.core.model import Model
from azureml.core.webservice import AksWebservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment

# Load the workspace
ws = Workspace.from_config()

# Register the new model
new_model = Model.register(
    workspace=ws,
    model_path="lightgbm_model.pkl",  # Path to the new model
    model_name="fraud-detection-model-v2",
    description="Optimized LightGBM model for fraud detection"
)

# Retrieve the existing AKS web service
aks_service = AksWebservice(workspace=ws, name="fraud-detection-api")

# Define the inference configuration (entry script and environment)
inference_config = InferenceConfig(
    entry_script="score.py",  # Path to your scoring script
    environment=Environment.get(workspace=ws, name="lightgbm-environment")  # Use the custom environment
)

# Redeploy the model to the existing AKS service
aks_service.update(
    models=[new_model],
    inference_config=inference_config
)

print("Optimized model deployed successfully.")

Registering model fraud-detection-model-v2
Optimized model deployed successfully.
